# Прогнозирование оттока пользователей
## Начальная обработка данных
Мне не понравилось то, как был сделан бейзлайн в предыдущем задании, поэтому я сделаю его сдесь еще раз.

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [9]:
X = pd.read_csv('data/orange_small_churn_data.train.txt')
y = pd.read_csv('data/orange_small_churn_labels.train.txt', header=None, names=['churn'])['churn']

In [40]:
X.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


In [12]:
X.shape, y.shape

((40000, 230), (40000,))

### Отбор признаков
Удалю все признаки, значения которых
1. NaN для всех пользвателей.
1. Значение которых одинаково для всех пользователей.
Дополнительно, получу список бинарных признаков.

In [72]:
nan_features = []
universal_features = []

for feature in X.columns:
    unique_values = X[feature].value_counts().shape[0]
    if unique_values == 0:
        nan_features.append(feature)
    elif unique_values == 1:
        universal_features.append(feature)

drop_features = nan_features + universal_features
print('Количество NaN признаков:', len(nan_features))
print('Количество признаков с одним значением:', len(universal_features))
print('Итого:', len(drop_features))

Количество NaN признаков: 18
Количество признаков с одним значением: 5
Итого: 23


In [73]:
X_droped_features = X.drop(drop_features, axis='columns')
print('Признаков до удаления: {}; признаков после удаления: {}; удалено: {}'.format(
    X.shape[1], X_droped_features.shape[1], X.shape[1] - X_droped_features.shape[1]))

Признаков до удаления: 230; признаков после удаления: 207; удалено: 23


### Обработка пропусков
Для бейзлайна заменю пропуски в числовых признаках средним; в категориальных — самым часто встречающимся значением.

In [74]:
num_features = [feat for feat in ['Var' + str(num) for num in np.linspace(1, 190, 190, dtype='int')]
                        if feat not in drop_features]
cat_features = [feat for feat in ['Var' + str(num) for num in np.linspace(191, 230, 40, dtype='int')]
                        if feat not in drop_features]
print('Числовых признаков:', len(num_features))
print('Категориальных признаков:', len(cat_features))

Числовых признаков: 173
Категориальных признаков: 34


In [75]:
means = X_droped_features[num_features].mean(axis=0)
X_num_filled = X_droped_features[num_features].fillna(means)
X_num_filled.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
0,11.003509,0.00497,429.328358,0.148235,249055.515021,3052.0,6.792126,47.487719,381346.56309,8.549254,...,3.823922e+06,0.0,1.401428e+06,80676.574553,7.419483,3.515789,16.05614,165.10831,270.585523,22637.159019
1,11.003509,0.00497,429.328358,0.148235,249055.515021,1813.0,7.000000,47.487719,381346.56309,8.549254,...,3.823922e+06,0.0,1.401428e+06,80676.574553,7.419483,3.515789,16.05614,165.10831,276.000000,22637.159019
2,11.003509,0.00497,429.328358,0.148235,249055.515021,1953.0,7.000000,47.487719,381346.56309,8.549254,...,3.823922e+06,0.0,1.401428e+06,80676.574553,7.419483,3.515789,16.05614,165.10831,270.585523,22637.159019
3,11.003509,0.00497,429.328358,0.148235,249055.515021,1533.0,7.000000,47.487719,381346.56309,8.549254,...,3.823922e+06,0.0,1.401428e+06,80676.574553,7.419483,3.515789,16.05614,165.10831,270.585523,22637.159019
4,11.003509,0.00497,429.328358,0.148235,249055.515021,686.0,7.000000,47.487719,381346.56309,8.549254,...,3.823922e+06,0.0,1.401428e+06,80676.574553,7.419483,3.515789,16.05614,165.10831,270.585523,22637.159019


In [78]:
modes = X_droped_features[cat_features].mode(axis='index').iloc[0]
X_cat_filled = X_droped_features[cat_features].fillna(modes)
X_cat_filled.head()

,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,...,Var219,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229
0,NESt0G8EIb,AERks4l,SEuy,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,yP09M03,smXZ,...,FzaX,hLKtJ9p,Al6ZaUT,vr93T2a,LM8l689qOp,ELof,fKCe,02N6s8f,xwM2aC7IdeMC0,am7c
1,P1WvyxLp3Z,2Knk1KF,SEuy,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,smXZ,...,FzaX,hN8KpA1,oslk,6hQ9lNX,LM8l689qOp,ELof,xb3V,RAYp,55YFVY9,mj86
2,FoxgUHSK8h,LrdZy8QqgUfkVShG,SEuy,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,yP09M03,smXZ,...,FzaX,4UxGlow,zCkv,catzS2D,LM8l689qOp,ELof,FSa2,ZI9m,ib5G6X1eUxUn6,mj86
3,vNEvyxLp3Z,RO12,SEuy,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,yP09M03,smXZ,...,FzaX,ylCK5YS,oslk,e4lqvY0,LM8l689qOp,ELof,xb3V,RAYp,F2FyR07IdsN7I,am7c
4,4e7gUH7IEC,RO12,SEuy,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,yP09M03,smXZ,...,FzaX,PYpzAu9,oslk,MAz3HNj,LM8l689qOp,ELof,WqMG,RAYp,F2FyR07IdsN7I,am7c


### Кодирование категориальных признаков
Для бейзлайна применю самый обычный LabelEncoder.

In [91]:
encoders = {feature: LabelEncoder() for feature in X_cat_filled}
X_cat_encoded = pd.DataFrame()
for feature in encoders.keys():
    X_cat_encoded[feature] = encoders[feature].fit_transform(X_cat_filled[feature])
X_cat_encoded.head()

,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,...,Var219,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229
0,159,5,1,22,0,0,1961,1235,12891,1,...,11,2855,0,3696,0,0,14,0,29,0
1,168,0,1,22,0,126,743,3520,3019,1,...,11,2864,4,347,0,0,22,2,2,1
2,114,17,1,22,0,110,2693,3462,12891,1,...,11,328,6,2498,0,0,7,3,25,1
3,328,25,1,22,0,2,3619,1542,12891,1,...,11,3834,4,2602,0,0,22,2,8,0
4,38,25,1,22,0,205,3005,2495,12891,1,...,11,1653,4,1375,0,0,12,2,8,0


In [98]:
X_filled = pd.concat([X_num_filled, X_cat_encoded], axis=1)

In [99]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X_filled, y, test_size=0.2, stratify=y)

## Baseline
Обучу 3 модели на имеющихся данных.
1. Линейная регрессия.
1. Случайный лес.
1. Градиентный бустинг.

In [120]:
skf = StratifiedKFold(n_splits=5)

logit_baseline = RidgeClassifier()
logit_scores = {}
logit_scores['estimator'] = 'logit'
logit_scores['roc_auc'] = cross_val_score(logit_baseline,
                                          X_train,
                                          y_train,
                                          cv=skf,
                                          scoring='roc_auc').mean()
logit_pred = cross_val_predict(logit_baseline, X_train, y_train, cv=skf)
logit_report = classification_report(y_train, logit_pred, output_dict=True)
logit_scores['recall'] = logit_report['1']['recall']
logit_scores['precision'] = logit_report['1']['precision']
logit_scores

{'estimator': 'logit',
 'roc_auc': 0.653690790507168,
 'recall': 0.0,
 'precision': 0.0}

In [122]:
rf_baseline = RandomForestClassifier(n_estimators=100)
rf_scores = {}
rf_scores['estimator'] = 'rf'
rf_scores['roc_auc'] = cross_val_score(rf_baseline,
                                          X_train,
                                          y_train,
                                          cv=skf,
                                          scoring='roc_auc').mean()
rf_pred = cross_val_predict(rf_baseline, X_train, y_train, cv=skf)
rf_report = classification_report(y_train, rf_pred, output_dict=True)
rf_scores['recall'] = rf_report['1']['recall']
rf_scores['precision'] = rf_report['1']['precision']
rf_scores

{'estimator': 'rf',
 'roc_auc': 0.6614663611436199,
 'recall': 0.0,
 'precision': 0.0}

In [124]:
gboost_baseline = GradientBoostingClassifier(n_estimators=100)
gboost_scores = {}
gboost_scores['estimator'] = 'rf'
gboost_scores['roc_auc'] = cross_val_score(gboost_baseline,
                                          X_train,
                                          y_train,
                                          cv=skf,
                                          scoring='roc_auc').mean()
gboost_pred = cross_val_predict(gboost_baseline, X_train, y_train, cv=skf)
gboost_report = classification_report(y_train, gboost_pred, output_dict=True)
gboost_scores['recall'] = gboost_report['1']['recall']
gboost_scores['precision'] = gboost_report['1']['precision']
gboost_scores

{'estimator': 'rf',
 'roc_auc': 0.7281878611987511,
 'recall': 0.005879882402351953,
 'precision': 0.35}